# Construir banco de Armazenamento Local Hotmart

## Configuração e Importações

### Objetivo: Esse pipeline tem por objetivo normalizar os labels de pesquisa que estejam divergentes entre versões de pesquisa criados.

In [1]:
# Importações básicas
import calendar
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


# Load Buyers Data

#### Load Hotmart Data

In [2]:
DATA_DIR = '../data/interim'
HIST_FILENAME_PATTERN = 'hotmart_sales_history_{}.csv'  # por ano

MODES = ['LOCAL_CACHE', 'FULL_REFRESH', 'ONLY_CURRENT_MONTH']
MODE = MODES[0]  # altere conforme necessário

def get_last_complete_month(today=None):
    today = today or datetime.today()
    first_of_month = today.replace(day=1)
    last_month = first_of_month - timedelta(days=1)
    return last_month.year, last_month.month

def get_year_date_range(year, start_month=1, end_month=12):
    start_date = f"{year}-{start_month:02d}-01"
    last_day = calendar.monthrange(year, end_month)[1]
    end_date = f"{year}-{end_month:02d}-{last_day:02d}"
    return start_date, end_date

def get_sales_history_period(start_date, end_date):
    return flatten_list_to_df(hotmart.get_sales_history(start_date=start_date, end_date=end_date))

START_YEAR = 2023
today = datetime.today()
last_year, last_month = get_last_complete_month(today)
current_year = today.year
current_month = today.month

dfs = []

if MODE == 'FULL_REFRESH':
    # Baixa e salva dados completos de cada ano até o último mês COMPLETO
    for year in range(START_YEAR, last_year + 1):
        if year == last_year:
            # Último ano pode ser incompleto (até o último mês completo)
            start_date, end_date = get_year_date_range(year, 1, last_month)
        else:
            start_date, end_date = get_year_date_range(year)
        df = get_sales_history_period(start_date, end_date)
        dfs.append(df)
        filename = HIST_FILENAME_PATTERN.format(year)
        save_processed_data(df, filename, data_dir=DATA_DIR)
elif MODE == 'LOCAL_CACHE':
    # Lê cache anual, cria-o caso não exista (para dados completos só de até mês passado)
    for year in range(START_YEAR, last_year + 1):
        filename = HIST_FILENAME_PATTERN.format(year)
        try:
            df = load_raw_data(filename, data_dir=DATA_DIR)
            # No caso do último ano no cache, se não está até mês passado, pega parcial e reprocessa
            if year == last_year:
                # Filtrar apenas até o último mês completo
                end_date = f"{last_year}-{last_month:02d}-{calendar.monthrange(last_year, last_month)[1]}"
                # A coluna de data é 'purchase_approved_date' no formato timestamp (ms)
                df = df[
                    pd.to_datetime(df['purchase_approved_date'], unit='ms') <= pd.to_datetime(end_date)
                ]
        except FileNotFoundError:
            # Baixa ano inteiro, ou parte dele no último ano
            if year == last_year:
                start_date, end_date = get_year_date_range(year, 1, last_month)
            else:
                start_date, end_date = get_year_date_range(year)
            df = get_sales_history_period(start_date, end_date)
            save_processed_data(df, filename, data_dir=DATA_DIR)
        dfs.append(df)
else:  # ONLY_CURRENT_MONTH ou outro caso especial
    dfs = []  # Use apenas dados do mês atual

# Sempre busca dados do mês atual via API para atualiza
current_month_start = f"{current_year}-{current_month:02d}-01"
current_month_end = today.strftime("%Y-%m-%d")
df_current_month = flatten_list_to_df(
    hotmart.get_sales_history(start_date=current_month_start, end_date=current_month_end)
)
dfs.append(df_current_month)

df_hotmart = pd.concat(dfs, ignore_index=True)

save_processed_data(df_hotmart, 'hotmart_sales_history.csv', data_dir=DATA_DIR)

### Load Hotmart Students


In [3]:
df_hotmart_students = flatten_list_to_df(hotmart.get_students(subdomain=CRONOGRAMA_SUBDOMAIN))

save_processed_data(df_hotmart_students, 'hotmart_students.csv', data_dir='../data/interim')

### Load TMB Data

In [4]:
df_tmb = load_raw_data('tmb.csv', sep=';')

"""
Cell generated by Data Wrangler.
"""
def clean_data(df_tmb):
    # Drop columns: 'Produtor', 'Pedido' and 29 other columns
    df_tmb = df_tmb.drop(columns=['Produtor', 'Pedido', 'Endereço completo', 'Logradouro', 'Número', 'Bairro', 'Complemento', 'CEP', 'Estado', 'Cidade', 'País', 'Status', 'Status Financeiro', 'Status Cancelamento', 'Ticket (R$)', 'Modalidade de Contrato', 'Criado Em', 'Data Efetivado', 'Data Cancelado', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_content', 'utm_last_source', 'utm_last_medium', 'utm_last_campaign', 'utm_last_content', 'Renovação automatica', 'Data de Renovação Prevista', 'Renovação enviada', 'Oferta'])
    # Drop column: 'Cliente CPF'
    df_tmb = df_tmb.drop(columns=['Cliente CPF'])
    # Rename column 'Produto' to 'offer_code'
    df_tmb = df_tmb.rename(columns={'Produto': 'offer_id', "Cliente Nome": "name", "Cliente Email": "email", "Telefone": "phone"})

    df_tmb['phone'] = process_phone_string(df_tmb, 'phone')
    
    return df_tmb

df_tmb = clean_data(df_tmb.copy())

save_processed_data(df_tmb, 'tmb_buyers_data.csv', data_dir='../data/interim')

In [5]:
TMB_HIST_FILENAME_PATTERN = 'tmb_pedidos_history_{}.csv'

dfs_tmb = []

if MODE == 'FULL_REFRESH':
    # Baixa e salva dados completos de cada ano até o último mês completo
    for year in range(START_YEAR, last_year + 1):
        if year == last_year:
            # Último ano: até o último mês completo
            start_date, end_date = get_year_date_range(year, 1, last_month)
        else:
            start_date, end_date = get_year_date_range(year)
        df = pd.DataFrame(tmb.get_pedidos(data_inicio=start_date, data_final=end_date, page_size=100))
        dfs_tmb.append(df)
        filename = TMB_HIST_FILENAME_PATTERN.format(year)
        save_processed_data(df, filename, data_dir=DATA_DIR)
elif MODE == 'LOCAL_CACHE':
    for year in range(START_YEAR, last_year + 1):
        filename = TMB_HIST_FILENAME_PATTERN.format(year)
        try:
            df = load_raw_data(filename, data_dir=DATA_DIR)
            # No caso do último ano, garante filtro apenas até o último mês completo
            if year == last_year and 'criado_em' in df.columns and not df.empty:
                end_date = f"{last_year}-{last_month:02d}-{calendar.monthrange(last_year, last_month)[1]}"
                # Conversão explícita de timezone para evitar TypeError
                # 1. Garante que 'criado_em' é datetime com no timezone.
                # 2. Remove timezone antes da comparação
                criado_em_dt = pd.to_datetime(df['criado_em'], errors='coerce').dt.tz_localize(None)
                end_date_dt = pd.to_datetime(end_date)
                df = df[criado_em_dt <= end_date_dt]
        except FileNotFoundError:
            if year == last_year:
                start_date, end_date = get_year_date_range(year, 1, last_month)
            else:
                start_date, end_date = get_year_date_range(year)
            df = pd.DataFrame(tmb.get_pedidos(data_inicio=start_date, data_final=end_date, page_size=100))
            save_processed_data(df, filename, data_dir=DATA_DIR)
        dfs_tmb.append(df)
else:  # ONLY_CURRENT_MONTH ou qualquer outro modo: só mês atual
    dfs_tmb = []

# Sempre busca o mês atual via API para atualizar (se não for só current month, concatena aos anteriores)
current_month_start = f"{current_year}-{current_month:02d}-01"
current_month_end = today.strftime("%Y-%m-%d")
df_tmb_current_month = pd.DataFrame(
    tmb.get_pedidos(data_inicio=current_month_start, data_final=current_month_end, page_size=100)
)
dfs_tmb.append(df_tmb_current_month)

# Concatena tudo e salva dataset total; garante coluna de data criada_em existe
df_tmb_api = pd.concat(dfs_tmb, ignore_index=True)
if 'criado_em' not in df_tmb_api.columns:
    df_tmb_api['criado_em'] = pd.NaT  # placeholder caso não venha nenhum dado

save_processed_data(df_tmb_api, 'tmb_pedidos_data.csv', data_dir=DATA_DIR)

### Load Generic Buyers Data

In [6]:
df_generic_buyers = db.execute_query_from_file(
    'select_generic_sales',
    params={"product_name": "Cronograma dos Fluentes"}
)

df_generic_buyers['phone'] = df_generic_buyers['phone'].apply(lambda x: str(int(x)) if isinstance(x, float) else x)

df_generic_buyers['phone'] = process_phone_string(df_generic_buyers, 'phone')

save_processed_data(df_generic_buyers, 'generic_buyers_data.csv', data_dir='../data/interim')
